In [2]:
import pandas as pd
import os
from bib_handling_code.processbib import read_bibfile
from bib_handling_code.processbib import save_to_file

C:\Users\keeli\anaconda3\envs\lit-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def from_bib_to_csv(diag_bib_raw):
    bib_data = []
    bib_columns = ['bibkey', 'type', 'title', 'authors', 'doi', 'gs_citations', 'journal', 'year', 'all_ss_ids', 'pmid']
    
    for bib_entry in diag_bib_raw:
        if bib_entry.type == 'string':
            continue

        bibkey = bib_entry.key
        bib_type = bib_entry.type
        fields = bib_entry.fields
        
        bib_authors = fields.get('author', '').strip('{}')
        bib_title = fields.get('title', '').strip('{}')
        bib_doi = fields.get('doi', '').strip('{}')
        bib_gscites = fields.get('gscites', '').strip('{}')
        bib_journal = fields.get('journal', '').strip('{}')
        bib_year = fields.get('year', '').strip('{}')
        bib_all_ss_ids = fields.get('all_ss_ids', '').strip('{}')
        bib_pmid = fields.get('pmid', '').strip('{}')
        
        bib_data.append([bibkey, bib_type, bib_title, bib_authors, bib_doi, bib_gscites, bib_journal, bib_year, bib_all_ss_ids, bib_pmid])

    df_bib_data = pd.DataFrame(bib_data, columns=bib_columns)
    return df_bib_data

# Read in the LIVE bib file and the updated bib file we want to merge it with

In [4]:
path_diag_bib = os.path.join('..', 'diag.bib')
diag_bib_raw = read_bibfile(None, path_diag_bib)
# convert the live bib file to csv format
bibfile_as_csv = from_bib_to_csv(diag_bib_raw)
path_new_bib = os.path.join('script_data/', 'diag_ss_tmp_new.bib')
updated_bib_raw = read_bibfile(None, path_new_bib)
# convert the updated bibfile to csv format
newbibfile_as_csv = from_bib_to_csv(updated_bib_raw)

In [5]:
new_bibkeys = newbibfile_as_csv['bibkey'].tolist()

In [6]:
old_bibkeys = bibfile_as_csv['bibkey'].tolist()

In [7]:
def get_entry(bibfile, bibkey_toupdate):
    for entry in bibfile:
        bibkey = entry.key
        if bibkey == bibkey_toupdate:
            return entry

In [8]:
# Update existing bibitems
"""KM: I propose that the updated bib file is the one we will keep and we only need to check if it misses anything that is in the original
for entry in diag_bib_raw:
    if entry.type == 'string':
        continue
    bibkey = entry.key
    if bibkey in new_bibkeys:
        corresponding_entry = get_entry(updated_bib_raw, bibkey)
        if 'all_ss_ids' in corresponding_entry.fields:
            entry.fields['all_ss_ids'] = corresponding_entry.fields['all_ss_ids']
        if 'pmid' in corresponding_entry.fields:
            entry.fields['pmid'] = corresponding_entry.fields['pmid']
"""

"KM: I propose that the updated bib file is the one we will keep and we only need to check if it misses anything that is in the original\nfor entry in diag_bib_raw:\n    if entry.type == 'string':\n        continue\n    bibkey = entry.key\n    if bibkey in new_bibkeys:\n        corresponding_entry = get_entry(updated_bib_raw, bibkey)\n        if 'all_ss_ids' in corresponding_entry.fields:\n            entry.fields['all_ss_ids'] = corresponding_entry.fields['all_ss_ids']\n        if 'pmid' in corresponding_entry.fields:\n            entry.fields['pmid'] = corresponding_entry.fields['pmid']\n"

In [9]:
bibkeys_not_in_updated = set(old_bibkeys)-set(new_bibkeys)
print(bibkeys_not_in_updated)

{'Brou23', 'Twil23c', 'Hump23', 'Anto23a', 'Aswo23', 'Hend23b'}


In [10]:
# bibkeys_toadd = set(new_bibkeys)-set(old_bibkeys)
# df_to_add = newbibfile[newbibfile['bibkey'].isin(bibkeys_toadd)]

In [11]:
# bibkeys_toadd

In [13]:
for entry in diag_bib_raw:
    bibkey = entry.key
    if bibkey in bibkeys_not_in_updated:
        print('appending entry', bibkey)
        updated_bib_raw.append(entry)

appending entry Twil23c
appending entry Hend23b
appending entry Anto23a
appending entry Brou23
appending entry Hump23
appending entry Aswo23


In [ ]:
#csv=from_bib_to_csv(diag_bib_raw)

In [14]:
path_diag_bib = os.path.join('..', 'diag_orig_and_ss_merged.bib')
save_to_file(updated_bib_raw, path_diag_bib)